- 기존 추천 시스템은 구매한 아이템을 기반으로 구매하지 않은 아이템 추천
- but, 우리는 구매한 아이템도 추천해야한다.
- 그러면 어떻게?
  - 기존 NCF, Glocal-K 등 방식은 무리가 있다.
    - sequence 정보가 들어가지 않기 때문
  - Sequence 정보가 들어가야 할 것 같다.
    - 하지만 sequence의 기준은 구매 이력이 아닌 시간으로 해야할 것 같다.
    - 적어도 하루 단위 이상으로 해야할 듯!
      - 하루는 의미 없을 것 같음(재방문율 한번 분석해보자! 아마 없을 것 같다,,,)
      - 또한 이 단위가 너무 작으면 sequence가 너무 길어져,,
        - Long Range Dependence!!!
        - 차원의 저주 및 계산 시간 증가
        - **적절한 window 크기**를 정하면 문제가 적을 것 같다!!!
- 하지만 구매 안한 시간이 더 많을테니 sparse해질 듯?
  - 이것 해결 방안?
  - 구매 안한 부분도 token으로 채워줘야 함: 따로 embedding 구해야겠지?
    - 이건 이것만의 의미가 있을 것이다.


- ㅁㅊ 아이템 구매 이력이 아닌 시간 간격으로 sequence 모델링한 논문이 있네?
  - TiSASRec: Time Interval Aware Self-Attention for Sequential Recommendation
  - GRU4Rec+: Session Based Rec
  - MARank가 2020 당시 SOTA? : Multi-Order Attentive Ranking Model for Sequential Recommendation

- 추천 시스템
  - Glocal-K
    - AE 사전학습(input: Rating Matrix) 후 fine-tuning
      - 저차원으로 축소: 유저-아이템 관계 비선형성으로 학습
    - local kernel, global kernel
      - local kernel: AE 사전 훈련
        - $r'$: AE output, $LK$: RBF kernel(U,V) output
        - $W \dot LK$가 encoder의 가중치로 사용됨
      - global kernel은 CNN 구조에서 영감 받음
        - $\mu_i$: $r'_i$를 avgpool, $k_i$: 그냥 가중치
        - $GK = \Sigma \mu_i \cdot k_i$: global kernel output
        - $GK$로 fine-tuning
  - Bert4Rec
    - $U$: user vector, $V$: item vector
    - $S = {v_1^{(u)}, v_2^{(u)}, \dots, v_{n_u}^{(u)}}$: 유저가 해당 sequence에 거래한 아이템 v의 집합
      - 이 $S$가 모델의 input
    - input embedding은 Lookup table 참조
    - Positional embedding을 학습
    - BERT 구조 사용(Transformer Encoder)
      - Bi-directional 효과!! 매우 좋음
    - Cloze task(MLM)로 학습(sliding window 안써요\~~)
      - 학습 비용이 크지 않다는 장점(1개의 데이터에 여러 개의 마스킹 있기 때문)
      - 근데 학습할 때 그 다음 아이템의 영향도 받는데 이건 괜찮나 모르겠당~
        - 만약 이걸로 태클 걸면 학습 과정에서 item의 embedding 값을 학습하게 되는데, 그 다음 아이템의 영향을 받으면 해당 embedding을 사용한 test에서더욱 좋다. 라고 말하면 어떨지?         - 영 꺼림직하면 GPT(Transformer Encoder)을 사용하던지~
    - test 시에는 마지막 sequence에 \[Mask\] 토큰 추가하여 예측

### 실험 설계
- 암시적 피드백이 맞는듯
  - 우리 데이터에 score가 없음
- 평가지표
  - HR, NDCG
  - K = ??
    - 이 K에 따라서 마케팅 전략 수행하면 좋을 듯